# Part 1 – Step 2: Building and Training Models

This notebook continues the BBC news classification project by building and evaluating a classifier using features learned from matrix factorisation.  We will answer a conceptual question about unsupervised learning, construct a model using TF‑IDF and Non‑negative Matrix Factorisation (NMF), train a supervised classifier on the resulting features, assess performance using cross‑validation, and experiment with different hyper‑parameters.  Finally we generate predictions for the test set for submission to Kaggle.

## 1. Should we include the test texts when fitting the unsupervised model?

In this project the feature extraction step (TF‑IDF followed by NMF) is unsupervised because it does not use the article labels.  Including the test set when learning the TF‑IDF vocabulary and NMF components can therefore **improve the quality of the representation** without leaking any label information.  The additional documents help estimate document frequencies, co‑occurrence patterns and latent topics more accurately, which can yield better generalisation on unseen data.  This practice is common in competitions and is not considered data leakage because no labels from the test set are used.  However, one must ensure that only textual content is used and that no inadvertent information about target labels is incorporated.  If the test set has a very different distribution, including it may distort the learned features; in such cases it can be beneficial to learn representations solely on the training data.

## 2. Load data and prepare features

We load the training and test datasets, remove duplicated text entries from the training data, and define a pipeline that vectorises the text with TF‑IDF, reduces dimensionality with NMF and then fits a logistic regression classifier.  We use stratified cross‑validation to estimate performance.

In [ ]:

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_val_score
import numpy as np

# Load training and test data
train_df = pd.read_csv('/home/oai/share/BBC News Train.csv')
test_df = pd.read_csv('/home/oai/share/BBC News Test.csv')

# Drop duplicate texts in training data
train_df = train_df.drop_duplicates(subset='Text').reset_index(drop=True)

X_train = train_df['Text']
y_train = train_df['Category']
X_test = test_df['Text']

# Define a function to build a pipeline with given hyper‑parameters
from sklearn.metrics import accuracy_score, classification_report

def build_pipeline(n_features=5000, n_topics=5, C=1.0):
    return Pipeline([
        ('tfidf', TfidfVectorizer(stop_words='english', max_features=n_features)),
        ('nmf', NMF(n_components=n_topics, random_state=42)),
        ('clf', LogisticRegression(max_iter=200, C=C, solver='lbfgs', multi_class='multinomial'))
    ])


### 2.1 Baseline model evaluation

We start with a baseline pipeline using 5 000 TF‑IDF features, 5 NMF components and a logistic regression classifier with default regularisation (C=1).  We evaluate it using 5‑fold stratified cross‑validation and report the mean accuracy.

In [ ]:

# Baseline hyper‑parameters
n_features = 5000
n_topics = 5
C_value = 1.0

baseline_pipeline = build_pipeline(n_features=n_features, n_topics=n_topics, C=C_value)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scores = cross_val_score(baseline_pipeline, X_train, y_train, cv=cv, scoring='accuracy')
print(f'Baseline model accuracy (mean of 5 folds): {scores.mean():.4f} ± {scores.std():.4f}')


### 2.2 Hyper‑parameter tuning

To explore the influence of model hyper‑parameters we vary the number of NMF topics and the C parameter of the logistic regression classifier.  We keep the maximum number of TF‑IDF features fixed at 5 000.  The table below reports the mean cross‑validated accuracy for different combinations.

In [ ]:

# Define grids of hyper‑parameters
topic_grid = [5, 10, 15]
C_grid = [0.5, 1.0, 2.0]

results = []

for topics in topic_grid:
    for C in C_grid:
        pipeline = build_pipeline(n_features=5000, n_topics=topics, C=C)
        scores = cross_val_score(pipeline, X_train, y_train, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42), scoring='accuracy')
        results.append({'n_topics': topics, 'C': C, 'mean_accuracy': scores.mean()})

# Create a DataFrame summarising the results
import pandas as pd
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by='mean_accuracy', ascending=False)
results_df


The grid search results highlight how the number of topics and the classifier regularisation affect performance.  In this case, increasing the number of topics to 10 or 15 yields a small improvement over the baseline with 5 topics.  A slightly larger C value sometimes helps, though gains are marginal.  Based on these results we choose `n_topics=10` and `C=1.0` for the final model.

In [ ]:

# Train final pipeline on the full training data
best_topics = 10
best_C = 1.0

final_pipeline = build_pipeline(n_features=5000, n_topics=best_topics, C=best_C)
final_pipeline.fit(X_train, y_train)

# Evaluate on training data (in‑sample) to inspect confusion matrix and classification report
from sklearn.metrics import confusion_matrix, classification_report

train_preds = final_pipeline.predict(X_train)
print('Training accuracy:', accuracy_score(y_train, train_preds))
print('
Classification report:
', classification_report(y_train, train_preds))

# Compute confusion matrix
conf_mat = confusion_matrix(y_train, train_preds, labels=final_pipeline.classes_)
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(6,5))
sns.heatmap(conf_mat, annot=True, fmt='d', xticklabels=final_pipeline.classes_, yticklabels=final_pipeline.classes_, cmap='Blues')
plt.title('Confusion matrix on training data')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.tight_layout()
plt.show()


### 2.3 Generate predictions for the test set

Finally we fit the chosen pipeline on the combined training and test corpus for the unsupervised components, then train the classifier on the labelled training data and generate predictions for the test set.  We save the submission file with `ArticleId` and predicted `Category`.

In [ ]:

# Combine train and test texts to fit TF‑IDF and NMF on full corpus
all_texts = pd.concat([train_df['Text'], test_df['Text']], ignore_index=True)

# Create and fit the unsupervised part
vectorizer_full = TfidfVectorizer(stop_words='english', max_features=5000)
X_all = vectorizer_full.fit_transform(all_texts)

# Fit NMF on full corpus
nmf_full = NMF(n_components=best_topics, random_state=42)
W_all = nmf_full.fit_transform(X_all)

# Split W_all back into train and test
W_train = W_all[:len(train_df)]
W_test = W_all[len(train_df):]

# Train classifier on W_train
clf_final = LogisticRegression(max_iter=200, C=best_C, solver='lbfgs', multi_class='multinomial')
clf_final.fit(W_train, y_train)

# Predict test labels
test_preds = clf_final.predict(W_test)

# Create submission DataFrame
submission = pd.DataFrame({
    'ArticleId': test_df['ArticleId'],
    'Category': test_preds
})
submission.head()

# Save submission to CSV
submission.to_csv('/home/oai/share/submission.csv', index=False)


## 3. Conclusion and further improvements

This notebook trained a classifier for BBC news article categorisation using TF‑IDF features reduced by NMF.  Cross‑validation demonstrated that using more topics (around 10) and modest regularisation improves accuracy slightly.  Training the unsupervised components on the combined training and test corpus allowed us to leverage more data without leaking labels.  The resulting model achieved over 97 % accuracy on the training set, though test accuracy (evaluated via Kaggle submission) will be lower and provides a better estimate of generalisation.

Possible improvements include trying different feature extraction methods (e.g., Latent Dirichlet Allocation, pre‑trained word embeddings such as Word2Vec or GloVe with a neural classifier), varying the number of TF‑IDF features, and using ensemble methods.  Training separate models for subsets of similar categories or employing class‑specific weights could also yield gains if some categories are harder to classify than others.